#### Install Autogen Library

In [1]:
#!pip install pyautogen agentops

#### Import Libraries

In [1]:
import autogen
import os

from dotenv import load_dotenv
load_dotenv()

True

#### Define LLM Endpoint Configuration

In [2]:
config_list = [
    {
        "model": os.getenv("MODEL_NAME_BIG"),
        "api_type": "azure",
        "api_version": "2023-07-01-preview",
        "base_url": os.getenv("AZURE_ENDPOINT"),
        # "api_key": api_key
    }
]

In [ ]:
api_key = os.getenv("AZURE_OPENAI_API_KEY")
config_list[0]["api_key"] = api_key

In [4]:
llm_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list,
    "timeout": 120,
}

In [5]:
import agentops
agentops.init(api_key=os.getenv("AGENTOPS_API_KEY"))

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=05ff33a0-7fdc-41f0-bd93-626f25a91bab


UUID('05ff33a0-7fdc-41f0-bd93-626f25a91bab')

## Construct Agents

#### Define a user proxy agent

In [6]:
user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
    code_execution_config=False,
)

#### Define an Engineer agent

In [7]:
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    system_message="""Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
""",
)

#### Define an Scientist agent

In [8]:
scientist = autogen.AssistantAgent(
    name="Scientist",
    llm_config=llm_config,
    system_message="""Scientist. You follow an approved plan. You are able to categorize papers after seeing their abstracts printed. You don't write code.""",
)

#### Define an Planner agent

In [9]:
planner = autogen.AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
The plan may involve an engineer who can write code and a scientist who doesn't write code.
Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a scientist.
""",
    llm_config=llm_config,
)

#### Define an Executor agent

In [10]:
executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "paper",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

#### Define a Critic agent

In [11]:
critic = autogen.AssistantAgent(
    name="Critic",
    system_message="Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL.",
    llm_config=llm_config,
)

#### Define the group chat Manager agent

In [12]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, scientist, planner, executor, critic], messages=[], max_round=50
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

## Start Chat

In [13]:
user_proxy.initiate_chat(
    manager,
    message="""
find and summarize the latest research in using foundational AI in Biometrics. create a markdown table of different papers and domains.
""",
)

Admin (to chat_manager):


find and summarize the latest research in using foundational AI in Biometrics. create a markdown table of different papers and domains.


--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

### Plan

1. **Research Phase**:
   - **Scientist**: Conduct a literature review to find the latest research papers on the use of foundational AI in biometrics. This will involve searching through academic databases such as Google Scholar, IEEE Xplore, PubMed, and others.
   - **Scientist**: Summarize the key findings, methodologies, and results of each paper.

2. **Data Collection Phase**:
   - **Scientist**: Collect the following details for each paper:
     - Title
     - Authors
     - Year of Publication
     - Journal/Conference
     - Domain (e.g., facial recognition, fingerprint analysis, voice recognition, etc.)
     - Key Findings
     - Methodologies Used
     - Results

3. **Data O

ChatResult(chat_id=None, chat_history=[{'content': '\nfind and summarize the latest research in using foundational AI in Biometrics. create a markdown table of different papers and domains.\n', 'role': 'assistant'}, {'content': "### Plan\n\n1. **Research Phase**:\n   - **Scientist**: Conduct a literature review to find the latest research papers on the use of foundational AI in biometrics. This will involve searching through academic databases such as Google Scholar, IEEE Xplore, PubMed, and others.\n   - **Scientist**: Summarize the key findings, methodologies, and results of each paper.\n\n2. **Data Collection Phase**:\n   - **Scientist**: Collect the following details for each paper:\n     - Title\n     - Authors\n     - Year of Publication\n     - Journal/Conference\n     - Domain (e.g., facial recognition, fingerprint analysis, voice recognition, etc.)\n     - Key Findings\n     - Methodologies Used\n     - Results\n\n3. **Data Organization Phase**:\n   - **Engineer**: Create a ma

In [14]:
agentops.end_session("Success")

🖇 AgentOps: This run's cost $0.338910
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=05ff33a0-7fdc-41f0-bd93-626f25a91bab
